# Import

In [1]:
import pandas as pd
import numpy as np
import sqlite3

# Neo4J Queries

    - provider_nodes
        - organization_nodes
        - taxonomy_nodes
    - hospital_nodes
        - organization_nodes
        - taxonomy_nodes
    - referral_relationships

## provider_nodes
    - entity type 1
    - any cbsa
    - referred to an entity type 2 in cbsa 34980
        - with at least 50 transactions and
        - with less than 50 avg wait time

In [2]:
%%time
# referrals from providers (entity 1) to metro nashville groups (entity 2) 
# having over 50 transactions and under 50 average wait time
query = """
        
        WITH metro_entity_2 AS (
            SELECT *
            FROM profile
            INNER JOIN taxonomy
            ON profile.taxonomy_code = taxonomy.code
            WHERE provider_business_practice_location_address_postal_code IN (
                SELECT zip
                FROM zip_cbsa
                WHERE cbsa = '34980'
                )
                AND entity_type_code = 2.0
            ),
        referrals50 AS (
            SELECT *
            FROM referrals
            WHERE average_day_wait < 50
            AND transaction_count >= 50
        ),
        qualifying_refs AS (
            SELECT *
            FROM metro_entity_2
            INNER JOIN referral_to
            ON metro_entity_2.npi = referral_to.to_npi
            INNER JOIN referrals50
            USING (referral_id)
        ),
        entity1 AS (
            SELECT npi
            FROM profile
            INNER JOIN taxonomy
            ON profile.taxonomy_code = taxonomy.code
            WHERE entity_type_code = 1.0
        )
        SELECT 
            to_npi,
            referral_id,
            qf.entity_type_code,
            qf.taxonomy_code,
            classification,
            specialization,
            qf."provider_organization_name_(legal_business_name)" AS organization,
            qf.provider_first_line_business_practice_location_address AS address_line1,
            qf.provider_second_line_business_practice_location_address AS address_line2,
            qf.provider_business_practice_location_address_city_name AS city,
            qf.provider_business_practice_location_address_state_name AS state,
            qf.provider_business_practice_location_address_postal_code AS zip
        FROM qualifying_refs AS qf
        INNER JOIN referral_from
        USING (referral_id)
        INNER JOIN entity1
        ON referral_from.from_npi = entity1.npi;

"""
with sqlite3.connect('../data/hopteam.sqlite') as db:
    groups = pd.read_sql(query, db)

Wall time: 1min 7s


In [3]:
%%time
# profiles of all providers
# with referrals to metro nashville groups (entity 2) having
# referrals of over 50 transactions and under 50 average wait time
query = """
        
        WITH metro_entity_2 AS (
            SELECT *
            FROM profile
            WHERE provider_business_practice_location_address_postal_code IN (
                SELECT zip
                FROM zip_cbsa
                WHERE cbsa = '34980'
                )
                AND entity_type_code = 2.0
            ),
        referrals50 AS (
            SELECT *
            FROM referrals
            WHERE average_day_wait < 50
            AND transaction_count >= 50
        ),
        qualifying_refs AS (
            SELECT 
                referral_id
            FROM metro_entity_2
            INNER JOIN referral_to
            ON metro_entity_2.npi = referral_to.to_npi
            INNER JOIN referrals50
            USING (referral_id)
        ),
        entity1 AS (
            SELECT *
            FROM profile
            INNER JOIN taxonomy
            ON profile.taxonomy_code = taxonomy.code
            WHERE entity_type_code = 1.0
        )
        SELECT
            from_npi,
            referral_id,
            entity_type_code,
            "provider_last_name_(legal_name)" || ', ' || provider_first_name AS name,
            taxonomy_code,
            classification,
            specialization,
            "provider_organization_name_(legal_business_name)" AS organization,
            provider_first_line_business_practice_location_address AS address_line1,
            provider_second_line_business_practice_location_address AS address_line2,
            provider_business_practice_location_address_city_name AS city,
            provider_business_practice_location_address_state_name AS state,
            provider_business_practice_location_address_postal_code AS zip     
        FROM qualifying_refs
        INNER JOIN referral_from
        USING (referral_id)
        INNER JOIN entity1
        ON referral_from.from_npi = entity1.npi;

"""

with sqlite3.connect('../data/hopteam.sqlite') as db:
    providers = pd.read_sql(query, db)

Wall time: 20.2 s


In [4]:
%%time
# referrals from providers (entity 1) to metro nashville groups (entity 2) 
# having over 50 transactions and under 50 average wait time
query = """
        
        WITH metro_entity_2 AS (
            SELECT *
            FROM profile
            WHERE provider_business_practice_location_address_postal_code IN (
                SELECT zip
                FROM zip_cbsa
                WHERE cbsa = '34980'
                )
                AND entity_type_code = 2.0
            ),
        referrals50 AS (
            SELECT *
            FROM referrals
            WHERE average_day_wait < 50
            AND transaction_count >= 50
        ),
        qualifying_refs AS (
            SELECT
                to_npi,
                referral_id,
                patient_count,
                transaction_count,
                average_day_wait,
                std_day_wait  
            FROM metro_entity_2
            INNER JOIN referral_to
            ON metro_entity_2.npi = referral_to.to_npi
            INNER JOIN referrals50
            USING (referral_id)
        ),
        entity1 AS (
            SELECT *
            FROM profile
            INNER JOIN taxonomy
            ON profile.taxonomy_code = taxonomy.code
            WHERE entity_type_code = 1.0
        )
        SELECT
            from_npi,
            to_npi,
            referral_id,
            patient_count,
            transaction_count,
            average_day_wait,
            std_day_wait
        FROM qualifying_refs
        INNER JOIN referral_from
        USING (referral_id)
        INNER JOIN entity1
        ON referral_from.from_npi = entity1.npi;

"""

with sqlite3.connect('../data/hopteam.sqlite') as db:
    referrals = pd.read_sql(query, db)

Wall time: 18.5 s


## Explore the dfs

### Count unique npis and organizations

In [5]:
print('There are ', providers['from_npi'].nunique(), 'unique recommending providers.')
print('There are ', providers['organization'].nunique(), 'unique recommending provider organizations.')
print('There are ', groups['to_npi'].nunique(), 'unique entity 2 npis referred to.')
print('There are ', groups['organization'].nunique(), 'unique entity 2 organizations referred to.')

There are  17773 unique recommending providers.
There are  0 unique recommending provider organizations.
There are  1495 unique entity 2 npis referred to.
There are  1262 unique entity 2 organizations referred to.


### Look at General Acute Care Hospitals in Nashville
VANDERBILT UNIVERSITY MEDICAL CENTER 1601 23RD AVE S is for adult psychiatric outpatient services.

HCA HEALTH SERVICES OF TENNESSEE, INC. 2300 PATTERSON ST is TriStar Centennial.

HCA HEALTH SERVICES OF TENNESSEE, INC. 391 WALLACE RD is TriStar Southern Hills.

SAINT THOMAS WEST HOSPITAL 4220 HARDING RD is Ascension Saint Thomas West on West End.

SAINT THOMAS WEST HOSPITAL 2000 CHURCH ST is Ascension Saint Thomas Midtown.

SETON CORPORATION has same address as Saint Thomas. It's a subsidiary (and a parent of others):

    - Parent organizations: Saint Thomas Health, Brady Corporation
    - Subsidiaries: Baptist Home Care, Baptist Healthcare Group

In [6]:
# Check out General Acute Care Hospitals in Nashville
(groups
     .loc[(groups['classification'].str.contains('General Acute Care Hospital', na = False)) 
           & 
           (groups['city'].str.contains('NASHVILLE'))
           ,
          ['organization', 'classification', 'specialization','address_line1', 'city', 'zip']]
     .drop_duplicates()
     .sort_values('organization')
     .reset_index()
)

,index,organization,classification,specialization,address_line1,city,zip
0,33509,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,None,2300 PATTERSON ST,NASHVILLE,37203
1,49937,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,None,391 WALLACE RD,NASHVILLE,37211
2,45127,HTI MEMORIAL HOSPITAL CORPORATION,General Acute Care Hospital,None,3441 DICKERSON PIKE,NASHVILLE,37207
3,46760,NASHVILLE GENERAL HOSPITAL,General Acute Care Hospital,None,1818 ALBION ST,NASHVILLE,37208
4,43782,SAINT THOMAS WEST HOSPITAL,General Acute Care Hospital,None,4220 HARDING RD,NASHVILLE,37205
5,65635,SAINT THOMAS WEST HOSPITAL,General Acute Care Hospital,None,2000 CHURCH ST,NASHVILLE,37236
6,66249,SETON CORPORATION,General Acute Care Hospital,None,2000 CHURCH ST,NASHVILLE,37236
7,51105,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,None,1601 23RD AVE S,NASHVILLE,37212
8,59044,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,None,1211 MEDICAL CENTER DRIVE,NASHVILLE,37232


### Look at General Acute Care Hospitals in Nashville CBSA (34 total)


In [7]:
cities = ['NASHVILLE',
          'MURFREESBORO',
          'BRENTWOOD',
          'FRANKLIN',
          'HENDERSONVILLE',
          'GOODLETTSVILLE',
          'LA VERGNE',
          'SMYRNA',
          'MT. JULIET',
          'GALLATIN']
# not currently using the cities list above
hospitals = (groups
     .loc[(groups['classification'].str.contains('General Acute Care Hospital', na = False)) 
#            & 
#            (groups['city'].isin(cities))
           ,
          ]
     .drop_duplicates()
     .sort_values('organization')
)

hospitals

,to_npi,referral_id,entity_type_code,taxonomy_code,classification,specialization,organization,address_line1,address_line2,city,state,zip
7974,1871530832,159858289,2.0,282N00000X,General Acute Care Hospital,None,CENTRAL TENNESSEE HOSPITAL CORPORATION,111 HIGHWAY 70 E,None,DICKSON,TN,37055
7927,1871530832,108311749,2.0,282N00000X,General Acute Care Hospital,None,CENTRAL TENNESSEE HOSPITAL CORPORATION,111 HIGHWAY 70 E,None,DICKSON,TN,37055
7926,1871530832,108311748,2.0,282N00000X,General Acute Care Hospital,None,CENTRAL TENNESSEE HOSPITAL CORPORATION,111 HIGHWAY 70 E,None,DICKSON,TN,37055
7925,1871530832,106475420,2.0,282N00000X,General Acute Care Hospital,None,CENTRAL TENNESSEE HOSPITAL CORPORATION,111 HIGHWAY 70 E,None,DICKSON,TN,37055
7924,1871530832,104580694,2.0,282N00000X,General Acute Care Hospital,None,CENTRAL TENNESSEE HOSPITAL CORPORATION,111 HIGHWAY 70 E,None,DICKSON,TN,37055
...,...,...,...,...,...,...,...,...,...,...,...,...
10878,1265445506,7541090,2.0,282N00000X,General Acute Care Hospital,None,WILLIAMSON COUNTY HOSPITAL DISTRICT,4321 CAROTHERS PARKWAY,None,FRANKLIN,TN,37067
10877,1265445506,5204129,2.0,282N00000X,General Acute Care Hospital,None,WILLIAMSON COUNTY HOSPITAL DISTRICT,4321 CAROTHERS PARKWAY,None,FRANKLIN,TN,37067
10876,1265445506,5204128,2.0,282N00000X,General Acute Care Hospital,None,WILLIAMSON COUNTY HOSPITAL DISTRICT,4321 CAROTHERS PARKWAY,None,FRANKLIN,TN,37067
11066,1265445506,132420210,2.0,282N00000X,General Acute Care Hospital,None,WILLIAMSON COUNTY HOSPITAL DISTRICT,4321 CAROTHERS PARKWAY,None,FRANKLIN,TN,37067


### Look at all Vanderbilt organizations

In [8]:
(groups
     .loc[groups['organization'].str.contains('VANDERBILT', na = False),
          ['organization', 'classification', 'specialization', 'zip']]
     .drop_duplicates()
)

,organization,classification,specialization,zip
8700,VANDERBILT HEALTH AND WILLIAMSON MEDICAL CENTE...,Internal Medicine,None,37064
18178,VANDERBILT UNIVERSITY MEDICAL CENTER,Psychiatric Unit,None,37087
18182,VANDERBILT UNIVERSITY MEDICAL CENTER,Rehabilitation Unit,None,37087
18185,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,None,37087
26037,"VANDERBILT MAURY RADIATION ONCOLOGY, LLC",Clinic/Center,"Oncology, Radiation",37174
41143,VANDERBILT UNIVERSITY MEDICAL CENTER,Clinic/Center,End-Stage Renal Disease (ESRD) Treatment,37204
44951,"VANDERBILT IMAGING SERVICES, LLC",Clinic/Center,Radiology,37205
50958,VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP,Rehabilitation Hospital,None,37212
51038,"VANDERBILT IMAGING SERVICES, LLC",Radiology,Diagnostic Radiology,37212
51105,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,None,37212


### Look at all organizations with 'Hospital' in the name

In [9]:
(groups
     .loc[groups['classification'].str.contains('Hospital'), 
          ['organization', 'classification', 'specialization']]
     .drop_duplicates()
     .sort_values('classification')
     .reset_index()
)

,index,organization,classification,specialization
0,185,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,Critical Access
1,45127,HTI MEMORIAL HOSPITAL CORPORATION,General Acute Care Hospital,None
2,43782,SAINT THOMAS WEST HOSPITAL,General Acute Care Hospital,None
3,26270,"SAINT THOMAS STONES RIVER HOSPITAL, LLC",General Acute Care Hospital,None
4,25460,NORTHCREST MEDICAL CENTER,General Acute Care Hospital,None
5,24615,"SAINT THOMAS DEKALB HOSPITAL, LLC",General Acute Care Hospital,None
6,24000,SHELBYVILLE HOSPITAL COMPANY LLC,General Acute Care Hospital,None
7,22153,SAINT THOMAS RUTHERFORD HOSPITAL,General Acute Care Hospital,None
8,20701,"TRUSTPOINT HOSPITAL, LLC",General Acute Care Hospital,None
9,66649,MAURY REGIONAL HOSPITAL,General Acute Care Hospital,None


### Look at the referrals data

In [10]:
referrals

,from_npi,to_npi,referral_id,patient_count,transaction_count,average_day_wait,std_day_wait
0,1821080961,1013012616,18854880,91,238,9.924,40.266
1,1841282779,1013012616,131966153,94,236,7.695,36.447
2,1770575607,1013012616,146169993,115,320,2.725,18.756
3,1508804337,1902804271,320488,157,163,27.583,51.939
4,1538109103,1902804271,6754370,312,320,12.709,40.583
...,...,...,...,...,...,...,...
69422,1558355941,1669872735,30083148,31,89,23.101,31.695
69423,1023099074,1669872735,114150220,256,1309,0.214,5.207
69424,1184619124,1669872735,178563129,40,90,23.089,36.995
69425,1205931565,1669872735,184096138,35,54,1.315,6.815


### Look at all providers who recommending to Nashville CBSA from out of state

In [11]:
providers[providers['state'] != 'TN']

,from_npi,referral_id,entity_type_code,name,taxonomy_code,classification,specialization,organization,address_line1,address_line2,city,state,zip
22,1427591627,44512612,1.0,"BECKMAN, LAUREN",363L00000X,Nurse Practitioner,None,None,18 ELMS CT,None,HATTIESBURG,MS,39402
41,1457888976,70281243,1.0,"MOLINA, DANIELLE",363A00000X,Physician Assistant,None,None,400 COURT ST STE 100,None,CHARLESTON,WV,25301
52,1669456307,104646182,1.0,"JOHNSTON, EVAN",367500000X,"Nurse Anesthetist, Certified Registered",None,None,3600 RIVERS AVE,None,CHARLESTON,SC,29405
59,1043228935,118660569,1.0,"D'ANGELO, PAUL",2085R0202X,Radiology,Diagnostic Radiology,None,5655 HUDSON DR STE 210,ARIS RADIOLOGY,HUDSON,OH,44236
110,1669456307,103864784,1.0,"JOHNSTON, EVAN",367500000X,"Nurse Anesthetist, Certified Registered",None,None,3600 RIVERS AVE,None,CHARLESTON,SC,29405
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69399,1598994147,80210852,1.0,"HARRIS, JIM",207P00000X,Emergency Medicine,None,None,2500 N STATE ST,UMC DEPART MENT OF EMERGENCY MEDICINE,JACKSON,MS,39216
69409,1063598373,25237153,1.0,"BURGESS, BERNARD",208600000X,Surgery,None,None,3601 SW 160TH AVE,SUITE 250,MIRAMAR,FL,33027
69411,1598994147,81057240,1.0,"HARRIS, JIM",207P00000X,Emergency Medicine,None,None,2500 N STATE ST,UMC DEPART MENT OF EMERGENCY MEDICINE,JACKSON,MS,39216
69419,1386976207,211436182,1.0,"ROTH, JASON",2085R0202X,Radiology,Diagnostic Radiology,None,INTEGRA IMAGING PS,1200 WESTWOOD DRIVE,HAMILTON,MT,59840


# Build formula to cleanly build full address, add to dfs

In [21]:
def build_address(df):
    address = ((df['address_line1'] + ', ' + df['address_line2']).mask(pd.isna, df['address_line1'])
         + ', ' 
         + df['city'] 
         + ', ' 
         + df['state']
         + ' '
         + df['zip']
    )
    return address

7974            111 HIGHWAY 70 E, DICKSON, TN 37055
7927            111 HIGHWAY 70 E, DICKSON, TN 37055
7926            111 HIGHWAY 70 E, DICKSON, TN 37055
7925            111 HIGHWAY 70 E, DICKSON, TN 37055
7924            111 HIGHWAY 70 E, DICKSON, TN 37055
                            ...                    
10878    4321 CAROTHERS PARKWAY, FRANKLIN, TN 37067
10877    4321 CAROTHERS PARKWAY, FRANKLIN, TN 37067
10876    4321 CAROTHERS PARKWAY, FRANKLIN, TN 37067
11066    4321 CAROTHERS PARKWAY, FRANKLIN, TN 37067
10882    4321 CAROTHERS PARKWAY, FRANKLIN, TN 37067
Length: 9068, dtype: object

In [22]:
build_address(providers)

0                302 N CONGRESS BLVD, SMITHVILLE, TN 37166
1                302 N CONGRESS BLVD, SMITHVILLE, TN 37166
2                302 N CONGRESS BLVD, SMITHVILLE, TN 37166
3         3024 BUSINESS PARK CIR, GOODLETTSVILLE, TN 37072
4         3024 BUSINESS PARK CIR, GOODLETTSVILLE, TN 37072
                               ...                        
69422    270 EAST MAIN STREET, SUITE 200, GALLATIN, TN ...
69423    507 GORDONSVILLE HWY, SUITE 203, GORDONSVILLE,...
69424       133 HOSPITAL DR, SUITE 500, CARTHAGE, TN 37030
69425                       37 PALMER ST, CALAIS, ME 04619
69426    8 NEW MIDDLETON HWY, STE A, GORDONSVILLE, TN 3...
Length: 69427, dtype: object

In [23]:
providers['address'] = build_address(providers)
hospitals['address'] = build_address(hospitals)

# Combine relevant dfs to make full_refs

In [24]:
full_refs = (providers.merge(hospitals, 
                on = 'referral_id',
               suffixes = ('_provider', '_hospital'))
          .merge(referrals,
                on = ['referral_id', 'from_npi', 'to_npi'])
)

full_refs

,from_npi,referral_id,entity_type_code_provider,name,taxonomy_code_provider,classification_provider,specialization_provider,organization_provider,address_line1_provider,address_line2_provider,...,address_line1_hospital,address_line2_hospital,city_hospital,state_hospital,zip_hospital,address_hospital,patient_count,transaction_count,average_day_wait,std_day_wait
0,1790730448,13418843,1.0,"STAFFORD, JAMES",2085R0202X,Radiology,Diagnostic Radiology,None,210 25TH AVE N STE 1204,None,...,313 N MAIN ST,None,ASHLAND CITY,TN,37015,"313 N MAIN ST, ASHLAND CITY, TN 37015",65,67,24.552,50.253
1,1790762219,13418844,1.0,"GOODIN, ELLIS",2085R0202X,Radiology,Diagnostic Radiology,None,210 25TH AVE N STE 1204,None,...,313 N MAIN ST,None,ASHLAND CITY,TN,37015,"313 N MAIN ST, ASHLAND CITY, TN 37015",61,63,17.238,35.569
2,1801017116,13418846,1.0,"STEWART, BRANDY",363LF0000X,Nurse Practitioner,Family,None,313 N MAIN ST,None,...,313 N MAIN ST,None,ASHLAND CITY,TN,37015,"313 N MAIN ST, ASHLAND CITY, TN 37015",104,186,19.129,51.613
3,1811077712,17417315,1.0,"SHOEMAKER, BENJAMIN",207Q00000X,Family Medicine,None,None,302 N MAIN ST,None,...,313 N MAIN ST,None,ASHLAND CITY,TN,37015,"313 N MAIN ST, ASHLAND CITY, TN 37015",50,90,17.611,47.253
4,1821060526,19402345,1.0,"WATERS, RONALD",2085R0202X,Radiology,Diagnostic Radiology,None,210 25TH AVE N STE 1204,None,...,313 N MAIN ST,None,ASHLAND CITY,TN,37015,"313 N MAIN ST, ASHLAND CITY, TN 37015",62,66,12.697,29.350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9063,1396713715,214911952,1.0,"ARTHUR, SCOTT",207XX0005X,Orthopaedic Surgery,Sports Medicine,None,4323 CAROTHERS PKWY STE 201,None,...,1224 TROTWOOD AVE,None,COLUMBIA,TN,38401,"1224 TROTWOOD AVE, COLUMBIA, TN 38401",40,73,30.151,51.364
9064,1407141773,216796225,1.0,"KAYS, KELLY",207R00000X,Internal Medicine,None,None,1224 TROTWOOD AVE,None,...,1224 TROTWOOD AVE,None,COLUMBIA,TN,38401,"1224 TROTWOOD AVE, COLUMBIA, TN 38401",387,927,0.019,0.591
9065,1407141302,216796226,1.0,"SHEPHERD, JOANNA",207R00000X,Internal Medicine,None,None,1224 TROTWOOD AVE,None,...,1224 TROTWOOD AVE,None,COLUMBIA,TN,38401,"1224 TROTWOOD AVE, COLUMBIA, TN 38401",224,582,0.153,3.689
9066,1407384480,216796227,1.0,"WENINEGAR, BRITTANY",363LF0000X,Nurse Practitioner,Family,None,44 HUGHES RD STE 100,None,...,1224 TROTWOOD AVE,None,COLUMBIA,TN,38401,"1224 TROTWOOD AVE, COLUMBIA, TN 38401",149,190,47.326,55.828


In [26]:
full_refs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9068 entries, 0 to 9067
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   from_npi                   9068 non-null   int64  
 1   referral_id                9068 non-null   int64  
 2   entity_type_code_provider  9068 non-null   float64
 3   name                       9066 non-null   object 
 4   taxonomy_code_provider     9068 non-null   object 
 5   classification_provider    9068 non-null   object 
 6   specialization_provider    4614 non-null   object 
 7   organization_provider      0 non-null      object 
 8   address_line1_provider     9068 non-null   object 
 9   address_line2_provider     3361 non-null   object 
 10  city_provider              9068 non-null   object 
 11  state_provider             9068 non-null   object 
 12  zip_provider               9068 non-null   object 
 13  address_provider           9068 non-null   objec

In [27]:
full_refs[full_refs['name'].isna()]

,from_npi,referral_id,entity_type_code_provider,name,taxonomy_code_provider,classification_provider,specialization_provider,organization_provider,address_line1_provider,address_line2_provider,...,address_line1_hospital,address_line2_hospital,city_hospital,state_hospital,zip_hospital,address_hospital,patient_count,transaction_count,average_day_wait,std_day_wait
2093,1215064852,183898559,1.0,None,2084N0402X,Psychiatry & Neurology,Neurology with Special Qualifications in Child...,None,1222 TROTWOOD AVE,SUITE 101,...,1080 N ELLINGTON PKWY,None,LEWISBURG,TN,37091,"1080 N ELLINGTON PKWY, LEWISBURG, TN 37091",41,67,19.552,32.098
8973,1215064852,184505060,1.0,None,2084N0402X,Psychiatry & Neurology,Neurology with Special Qualifications in Child...,None,1222 TROTWOOD AVE,SUITE 101,...,1224 TROTWOOD AVE,None,COLUMBIA,TN,38401,"1224 TROTWOOD AVE, COLUMBIA, TN 38401",301,480,27.860,41.480


# Write full_refs to CSV for Neo4J

In [25]:
full_refs.to_csv('../data/neo4j/full_refs.csv', index = False)